In [ ]:
import utils
from data_acquisition import DataHandler
import numpy as np
from torch.utils.data import Dataset


# Configure logging for the pipeline
logger = utils.setup_logger(level='INFO')

In [ ]:
# print(utils.get_available_cities())
cities = ['Aachen', 'Frankfurt', 'CapeTown', 'Hamburg', 'Johannesburg', 'London', 'Montreal', 'Paris', 'Seoul', 'Singapore', 'Sydney']
datahandler = DataHandler(logger)

from data_preparation import DataPrepare, get_data_loaders
dataprepper = DataPrepare(datahandler, logger, patch_size=128)


In [ ]:
transformed_images = []

for city in cities[:2]:
    transformed_images.append(dataprepper.create_tensor_of_windows(city))

transformed_images_merged = np.concatenate(transformed_images, axis=0)
transformed_images_merged = np.transpose(transformed_images_merged, (0,3,1,2))
train_loader, test_loader = get_data_loaders(transformed_images_merged,train_ratio=0.8, batch_size=64)

In [ ]:
 for batch in train_loader:
        inputs = batch[:,:-1].to(torch.float32)
        labels = batch[:,-1].to(int)
        print(labels.max())
        break

In [ ]:
import torch.nn as nn
model = nn.Sequential(
    nn.Conv2d(6, 32, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(128, 1, kernel_size=1, padding=0),
    nn.Sigmoid())

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch


# Instantiate the model, loss function, and optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        inputs = batch[:,:-1].to(torch.float32)
        labels = batch[:,-1, np.newaxis].to(torch.float32)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


In [ ]:
import numpy as np
transformed_images_merged = np.concatenate(transformed_images, axis=0)

dataprepper.plot_sub_image(transformed_images_merged[3000]).show()

In [ ]:
datahandler.get_building_mask("Frankfurt")

In [ ]:


buildings = []
sat_images = []
building_masks = []

for city in cities: 
    buildings.append(datahandler.get_buildings(city))
    sat_images.append(datahandler.get_satellite_image(city))
    building_masks.append(datahandler.get_building_mask(city))

# Plot the expected results for the first city 
datahandler.plot(city[0])

In [ ]:
import data_preparation

for city in cities:
    data_preparation.create_tensor(city)

# Download

In [ ]:
# Download 

for city in cities: 
    sat_image = datahandler.get_satellite_image(city)
    mask = datahandler.get_building_mask(city)

# Plot the expected results for the first city 
datahandler.plot(city[0])